<a href="https://colab.research.google.com/github/2025-01-sookmyung-opensource/final_project/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# 1. 필요한 패키지 설치
!pip install -q streamlit pyngrok ultralytics gdown opencv-python pillow

In [19]:
# 2. pyngrok 임포트 및 인증 토큰 설정
from pyngrok import ngrok

# ngrok 인증 토큰 마스킹
ngrok.set_auth_token("2Np")

# 3. 기존에 열려있는 ngrok 세션 종료 (중복 방지)
ngrok.kill()

In [20]:
# 4. 학습된 YOLO 모델(best.pt) 다운로드 (Google Drive)
import gdown
gdown.download(id='1sl1dXyWuJiXjhHeacomWwfppsM57VJ85', output='best.pt', quiet=False)

# 5. 스트림릿 앱 코드 작성 (app.py로 저장)
code = '''
import streamlit as st
from PIL import Image
import numpy as np
import cv2
from ultralytics import YOLO

model = YOLO('best.pt')

st.set_page_config(page_title="🍱 식단 이미지 분석", layout="centered")
st.title("🍱 식단 이미지 분석기")
st.markdown("이미지를 업로드하면 YOLO 모델이 음식을 인식하고 결과를 표시합니다.")

uploaded_file = st.file_uploader("이미지를 업로드하세요", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption="업로드한 이미지", use_column_width=True)

    results = model.predict(np.array(image))

    detected = {}
    for result in results:
        class_ids = result.boxes.cls.cpu().numpy().astype(int)
        for cid in class_ids:
            name = model.names[cid]
            detected[name] = detected.get(name, 0) + 1

    st.subheader("🍽️ 인식된 항목:")
    if detected:
        for name, count in detected.items():
            st.write(f"- {name}: {count}개")
    else:
        st.write("아무것도 인식되지 않았습니다.")

    annotated = results[0].plot()
    annotated = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
    st.image(annotated, caption="YOLO 인식 결과", use_column_width=True)
'''

with open("app.py", "w") as f:
    f.write(code)

# 6. 스트림릿 서버 실행 (백그라운드)
!nohup streamlit run app.py > log.txt 2>&1 &

# 7. ngrok 터널 열기 (포트 8501)
public_url = ngrok.connect(addr="8501", proto="http")
print(f"✅ 접속 주소: {public_url}")

# 8. 로그 확인 (필요 시)
!tail -n 20 log.txt

Downloading...
From: https://drive.google.com/uc?id=1sl1dXyWuJiXjhHeacomWwfppsM57VJ85
To: /content/best.pt
100%|██████████| 6.26M/6.26M [00:00<00:00, 40.7MB/s]


✅ 접속 주소: NgrokTunnel: "https://d5f2-34-16-160-81.ngrok-free.app" -> "http://localhost:8501"


